In [ ]:
!pip install vectorbt

In [2]:
import vectorbt as vbt
import pandas as pd

In [8]:
testDays = 1  # 测试天数
testStartDate = pd.to_datetime('2025-07-01')
# print(testDays, testStartDate)

In [9]:
df = pd.read_csv('spy_minute.csv')
df.columns = [col.strip().lower() for col in df.columns]  # 标准化列名

# start from a specific date
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] >= testStartDate]  # 只保留从指定日期开始的数据
df = df.head(350 * testDays)

In [10]:
# 构造 datetime 索引
df['datetime'] = pd.to_datetime(df['date']) + pd.to_timedelta(df['minute'], unit='m')
df = df.set_index('datetime')
df = df.sort_index()

# 可选：删除无用字段
df = df[['open', 'high', 'low', 'close', 'volume']]

# 计算 rolling 高/低（5分钟）
rolling_high = df['high'].shift(1).rolling(5).max()
rolling_low = df['low'].shift(1).rolling(5).min()

# print(rolling_high.head(10))
# print(rolling_low.head(10))

# print last 10 rows of the DataFrame
# print(rolling_high.tail(10))
# print(rolling_low.tail(10))

In [11]:
# 策略进出信号
long_entry = df['close'] > rolling_high
short_entry = df['close'] < rolling_low

# 每日最后一分钟强制平仓
df['date'] = df.index.date
last_bar_idx = df.groupby('date').tail(1).index
force_exit = df.index.isin(last_bar_idx)

# 出场信号逻辑：交叉方向时平仓 + 每日收盘前清仓
long_exit = short_entry | force_exit
short_exit = long_entry | force_exit

In [ ]:

# 回测（方向切换时自动强平）
portfolio = vbt.Portfolio.from_signals(
    close=df['close'],
    entries=long_entry,
    exits=long_exit,
    short_entries=short_entry,
    short_exits=short_exit,
    size=1.0,
    direction='both',
    freq='1min',
    init_cash=100_000,
    fees=0.0005
)

# 输出统计和图表
print(portfolio.stats())
portfolio.plot().show()


Start                               2025-07-01 06:30:00
End                                 2025-07-01 12:19:00
Period                                  0 days 05:50:00
Start Value                                    100000.0
End Value                                  99980.461135
Total Return [%]                              -0.019539
Benchmark Return [%]                           0.218943
Max Gross Exposure [%]                         0.618747
Total Fees Paid                               17.598465
Max Drawdown [%]                               0.020275
Max Drawdown Duration                   0 days 05:28:00
Total Trades                                         29
Total Closed Trades                                  28
Total Open Trades                                     1
Open Trade PnL                                 -0.05885
Win Rate [%]                                   3.571429
Best Trade [%]                                 0.026006
Worst Trade [%]                               -0

/var/folders/sv/xpvmcdvj3mx3zt_nkc40m0wc0000gp/T/ipykernel_16920/25138966.py:2: UserWarning:

direction has no effect if short_entries and short_exits are set



AttributeError: module 'vectorbt' has no attribute 'OHLC'